In [2]:
import pdal 
import numpy as np
import matplotlib.pyplot as plt
import open3d as o3d
from scipy.spatial import KDTree
from scipy.sparse.csgraph import connected_components
import scipy.sparse as sp
import os
import json
import pyvista as pv
import time

from interessant import * # Bei Änderungen Kernel neu starten

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
o3d.__version__  

'0.19.0'

In [5]:
run = run24
#run = run14
# filename = interessant['OLA gleiche Höhe wie Gleis']

# Bahnsteig: 29; Gleis hohe Intensität: 11; Weiche B: 16; Unterirdischer Bhf: 20; Gleis weit abseits: 23; Betondeckel: 28; Zug run 14 A (in run24 Achszähler): 6; 
# Viele Gleise: 33; Anfang Weiche: 34; Weiche C: 38 OLA gleiche H: 35; Y: 37
key = list(interessant.keys())[0] 
filename = interessant[key]
print(key, filename)

filename = os.path.join(run, filename)
if not os.path.exists(filename):
    raise FileNotFoundError(filename)

Einfach 4473900_5335875.copc.laz


In [6]:
thresh = 8  # z.B. 5 oder 8
majority_tresh  = 0.5 # Erster Durchgang 0.3, bei "Gleis hohe Intensität" gibt 0.5 ein viel besseres Ergebnis

voxel_size = 1.0

voxel_size = 25 / 30
print("Voxel size:", voxel_size)

minimum_points = 50 # Erste Versuche mit 100, aber viel schwarz bei abseits liegenden Gleisen. 50 ist besser.
minimum_in_hood = 10
linearity_tresh = 0.98

intensity_threshold = 14500
downsample_radius = 0.3
neighborhood_radius = 0.5

Voxel size: 0.8333333333333334


In [7]:
pipeline = pdal.Pipeline([pdal.Reader(filename)])
pipeline.execute()
points = pipeline.arrays[0]

In [8]:
xyz = np.vstack((points['X'], points['Y'], points['Z'])).transpose()

In [9]:
points['Classification'] = 0 # Unclassified
RAIL = 20

In [10]:
maxp = xyz.max(axis=0)
minp = xyz.min(axis=0)
maxp, minp

(array([4.47365000e+06, 5.33800000e+06, 5.17640641e+02]),
 array([4.47363330e+06, 5.33797500e+06, 5.10004041e+02]))

In [11]:
voxels = xyz.copy()
voxels[:, :2] = ((xyz[:, :2] - minp[:2]) // voxel_size).astype(int)

In [12]:
# Anzahl der Voxel checken
np.ceil((maxp[:2] - minp[:2]) / voxel_size).astype(int)

array([21, 30])

In [13]:
from collections import defaultdict
voxel_dict = defaultdict(list)
index_dict = defaultdict(list)

# Füllen des Dictionaries
for idx, (point, voxel) in enumerate(zip(xyz, voxels)):
    voxel_key = tuple(voxel[:2])
    voxel_dict[voxel_key].append(point[2])
    index_dict[voxel_key].append(idx)

In [14]:
for key, z_values in voxel_dict.items():
    
    # Threshold on number of points in voxel
    if len(z_values) < minimum_points:
        continue

    indices = np.array(index_dict[key])
    z_values = np.array(z_values)
    ground_level = np.percentile(z_values, 10) # 10% Percentile
    # Check that there are almost no points 0.5 to 4.5 m above the ground
    # But allow for some noise
    # thresh = 3 # Der einfachheit halber oben
    count = ((z_values > ground_level + 0.5) & (z_values < ground_level + 4.5)).sum()

    if count <= thresh:
        # Look for points within 0.5 m above ground and get 98% percentile ODER 99.5
        mask = (z_values > ground_level) & (z_values < ground_level + 0.5)
        try:
            candidates_top = np.percentile(z_values[mask], 99.5)
        except IndexError:
            # Fails if there are no points in the masked array
            continue

        # Oude Elberink require the height difference > 0.1 m
        # And mark only the points 10 cm below the top as rail point candidates
        if candidates_top - ground_level > 0.1:
            mask = (z_values > candidates_top - 0.1) & (z_values < candidates_top + 0.05)

            # Also make sure these are only a minority of the points (otherwise it's a slope)
            if mask.sum() < majority_tresh * len(z_values):  # z.B. 0.3
                points['Classification'][indices[mask]] = RAIL

In [15]:
candidates = points[points["Classification"] == RAIL]
candidates.shape

(5415,)

In [16]:
# filters.outlier sets Classification to 7, filters.range removes the points with Classification 7

noise_filter = pdal.Filter("filters.outlier", method="statistical", mean_k=10, multiplier=2.0).pipeline(candidates) | pdal.Filter("filters.range", limits="Classification![7:7]")
print(noise_filter.toJSON())
noise_filter.execute()
candidates = noise_filter.arrays[0]
candidates.shape 

[{"type": "filters.outlier", "method": "statistical", "mean_k": 10, "multiplier": 2.0, "tag": "filters_outlier1"}, {"type": "filters.range", "limits": "Classification![7:7]", "tag": "filters_range1"}]


(5296,)

## Schreiben und lesen testen
### LAZ

In [17]:
start = time.time()
out_pipeline = pdal.Writer("test.laz").pipeline(candidates[['X', 'Y', 'Z', 'Intensity']])
out_pipeline.execute()
print("Time:", time.time() - start)

Time: 0.006429433822631836


In [18]:
os.path.getsize("test.laz")

24976

In [19]:
start = time.time()
pipeline = pdal.Pipeline([pdal.Reader("test.laz")])
pipeline.execute()
test = pipeline.arrays[0]
print("Time:", time.time() - start)


Time: 0.010869503021240234


In [20]:
# Ohne Anpassung von scale und offset gehen die Nachkommastellen bis auf die erste verloren
(test["X"] == candidates["X"]).all()

np.False_

### PLY

In [21]:
start = time.time()
out_pipeline = pdal.Writer("test.ply", storage_mode="little endian").pipeline(candidates[['X', 'Y', 'Z', 'Intensity']])
s = out_pipeline.execute()
print("Time:", time.time() - start)

Time: 0.004980802536010742


In [33]:
# Externe Platte

start = time.time()
out_pipeline = pdal.Writer("/media/riannek/minimax/gleis/temp/test.ply", storage_mode="little endian").pipeline(candidates[['X', 'Y', 'Z', 'Intensity']])
s = out_pipeline.execute()
print("Time:", time.time() - start)

Time: 0.0033385753631591797


In [22]:
s 

5296

In [23]:
os.path.getsize("test.ply")

137872

ply (binär) ist etwas schneller zu lesen/schreiben, deutlich größer, aber OK, alle Nachkommastellen sind da

In [24]:
start = time.time()
pipeline = pdal.Pipeline([pdal.Reader("test.ply")])
pipeline.execute()
test = pipeline.arrays[0]
print("Time:", time.time() - start)
test 

Time: 0.0027709007263183594


array([(4473645.7970327, 5337995.13721208, 510.25114071,  9766),
       (4473648.5562327, 5337991.89991208, 510.25364071, 12079),
       (4473648.5260327, 5337991.97131208, 510.16794071, 12593), ...,
       (4473641.3519327, 5337994.18871208, 510.23544071, 11565),
       (4473640.1381327, 5337997.44181208, 510.13594071, 12079),
       (4473644.9549327, 5337991.57071208, 510.16414071, 20560)],
      dtype=[('X', '<f8'), ('Y', '<f8'), ('Z', '<f8'), ('Intensity', '<u2')])

In [25]:
(test["X"] == candidates["X"]).all()

np.True_

### PCD
Auch hier gehen z.T. Kommastellen verloren

In [26]:
start = time.time()
out_pipeline = pdal.Writer("test.pcd", compression="binary").pipeline(candidates[['X', 'Y', 'Z', 'Intensity']])
out_pipeline.execute()
print("Time:", time.time() - start)

Time: 0.002439260482788086


In [27]:
os.path.getsize("test.pcd")

106077

In [28]:
start = time.time()
pipeline = pdal.Pipeline([pdal.Reader("test.pcd")])
pipeline.execute()
test = pipeline.arrays[0]
print("Time:", time.time() - start)
test 

Time: 0.0015687942504882812


array([(4473646. , 5337995. , 510.25112915,  9766.),
       (4473648.5, 5337992. , 510.25363159, 12079.),
       (4473648.5, 5337992. , 510.16793823, 12593.), ...,
       (4473641.5, 5337994. , 510.23544312, 11565.),
       (4473640. , 5337997.5, 510.13595581, 12079.),
       (4473645. , 5337991.5, 510.16415405, 20560.)],
      dtype=[('X', '<f8'), ('Y', '<f8'), ('Z', '<f8'), ('Intensity', '<f8')])

In [29]:
(test["X"] == candidates["X"]).all()

np.False_

In [30]:
os.path.join(
os.path.basename(filename).split(".")[0], ".ply") 

'4473625_5337975/.ply'

## Noisefilter und PLY in einer Pipeline

In [31]:
out_pipeline = pdal.Filter("filters.outlier", method="statistical", mean_k=10, multiplier=2.0).pipeline(candidates[['X', 'Y', 'Z', 'Intensity']]) | pdal.Filter("filters.range", limits="Classification![7:7]") | pdal.Writer("test.ply", storage_mode="little endian")
out_pipeline.execute()



5063

In [32]:
pipeline.metadata['metadata']['readers.copc']['srs']['json']['id']

KeyError: 'readers.copc'